In [1]:
import sys
sys.path.append('../src')
from ds_utils import *
from PIL import Image

In [2]:
import pandas as pd
df = pd.read_csv('/home/jcejudo/rd-img-classification-pilot/conversion_vocabularies.csv')

In [3]:
df.head(10)

,concept,getty_aat,dbpedia,wikidata,ef_entity_collection
0,building,http://vocab.getty.edu/aat/300004792,http://dbpedia.org/page/Building,http://www.wikidata.org/entity/Q41176,http://data.europeana.eu/concept/base/29
1,archaeological_site,http://vocab.getty.edu/aat/300266151,http://dbpedia.org/page/Archaeological_site,http://www.wikidata.org/entity/Q839954,NaN
2,cartoon,http://vocab.getty.edu/aat/300123430,http://dbpedia.org/page/Cartoon,http://www.wikidata.org/entity/Q627603,NaN
3,ceramics,http://vocab.getty.edu/aat/300151343,http://dbpedia.org/resource/Ceramic\n,http://www.wikidata.org/entity/Q45621,http://data.europeana.eu/concept/base/31
4,clothing,http://vocab.getty.edu/aat/300266639,http://dbpedia.org/page/Clothing,http://www.wikidata.org/entity/Q11460,NaN
5,costume_accessories,http://vocab.getty.edu/aat/300209273,NaN,http://www.wikidata.org/entity/Q1065579,NaN
6,drawing,http://vocab.getty.edu/aat/300033973,http://dbpedia.org/resource/Drawing,http://www.wikidata.org/entity/Q93184,http://data.europeana.eu/concept/base/35
7,map,http://vocab.getty.edu/aat/300028094,http://dbpedia.org/resource/Map,http://www.wikidata.org/entity/Q4006,http://data.europeana.eu/concept/base/43
8,furniture,http://vocab.getty.edu/aat/300037680,http://dbpedia.org/page/Furniture,http://www.wikidata.org/entity/Q14745,http://data.europeana.eu/concept/base/37
9,textile,http://vocab.getty.edu/aat/300231565,http://dbpedia.org/page/Textile,http://www.wikidata.org/entity/Q28823,http://data.europeana.eu/concept/base/55


In [83]:
df = pd.read_csv('/home/jcejudo/rd-img-classification-pilot/dataset_10000.csv')

In [84]:
df.shape

(132042, 5)

In [85]:
len(df.category.unique())

20

In [94]:
for cat in df.category.unique():
    print(cat,df.loc[df['category'] == cat].shape[0])

building 10000
ceramics 10000
drawing 10000
furniture 7478
jewellery 4765
map 10000
painting 10000
photograph 10000
postcard 10000
sculpture 10000
specimen 10000
tapestry 341
textile 10000
toy 8500
woodwork 1020
archaeological_site 1105
clothing 2032
costume_accessories 2166
inscription 434
weaponry 4201


In [2]:
#check downloaded images
data_dir = '/home/jcejudo/rd-img-classification-pilot/training_data_3000'
for cat in os.listdir(data_dir):
    print(cat, len(os.listdir(os.path.join(data_dir,cat))))

building 2996
toy 2893
map 2795
textile 2939
clothing 1974
woodwork 959
photograph 3000
ceramics 2971
inscription 434
costume_accessories 2159
furniture 2844
weaponry 2988
archaeological_site 690
painting 2991
specimen 2789
tapestry 325
drawing 1853
postcard 3000
sculpture 3000
jewellery 2958


In [3]:
#data wrangling

# get europeana context from labeled images
def replace_str(x,target,rep):
    return x.replace(target,rep)+'.jpg'

def get_fname(path):
    return os.path.split(path)[1]


def get_europeana_df(data):
    #get a df with europeana context
    df_root_path = '/home/jcejudo/rd-img-classification-pilot/data'
    df_path = os.path.join(df_root_path,data)
    df = pd.DataFrame(columns=['category', 'skos_concept', 'URI', 'ID', 'URL'])
    for fname in os.listdir(df_path):
        #print(fname)
        df = pd.concat((df,pd.read_csv(os.path.join(df_path,fname))))

    df['filename'] = df['ID'].apply(lambda x:replace_str(x,'/','__placeholder__'))
    df.pop('category')
    df['filename'].values

    #get dataframe for labeled files
    root_data_path = '/home/jcejudo/rd-img-classification-pilot/training_data/'
    data_path = os.path.join(root_data_path,data)
    file_df = path2DataFrame(data_path)
    file_df['filename'] = file_df['file_path'].apply(lambda x:get_fname(x))
    file_df.pop('file_path')
    file_df['filename'].values

    #merge by filename
    result = pd.merge(df, file_df, on="filename")
    return result


getty_df = get_europeana_df('getty')
ec_df = get_europeana_df('ec')
df = pd.concat((getty_df,ec_df))
print(df.shape)
print(df.columns)

#save csv
filename = 'dataset.csv'
dest_path = '/home/jcejudo/rd-img-classification-pilot/data'
dest_file_path = os.path.join(dest_path,filename)
df.to_csv(dest_file_path,index=False)


toy.csv
food.csv
painting.csv
map.csv
costume_accessories.csv
medal.csv
musical_instrument.csv
poster.csv
postcard.csv
inscription.csv
furniture.csv
archaeological_site.csv
sculpture.csv
building.csv
machinery.csv
photograph.csv
metalwork.csv
weaponry.csv
tapestry.csv
glassware.csv
clothing.csv
drawing.csv
jewellery.csv
tool.csv
ceramics.csv
toy.csv
painting.csv
map.csv
postcard.csv
furniture.csv
sculpture.csv
building.csv
photograph.csv
textile.csv
woodwork.csv
specimen.csv
tapestry.csv
drawing.csv
jewellery.csv
ceramics.csv
(18744, 6)
Index(['skos_concept', 'URI', 'ID', 'URL', 'filename', 'category'], dtype='object')
